In [0]:
import azure.cognitiveservices.speech as speechsdk

In [0]:
clave = "a9328ecbc77b4e53b8943a627ddd4cf0"
region = "northcentralus"

In [0]:
%fs cp dbfs:///mnt/azure/data/AUDIO_CALL_CENTER.wav file:///tmp

res3: Boolean = true

In [0]:
from pydub import AudioSegment
tiempoDeChunkMilisegundos = 5000

In [0]:
audio = AudioSegment.from_file("/tmp/AUDIO_CALL_CENTER.wav")

In [0]:
audioChunks = []

In [0]:
for i in range(0,len(audio),tiempoDeChunkMilisegundos):
  chunk = audio[i:(i+tiempoDeChunkMilisegundos)]
  archivoChunk = "/tmp/chunk_"+str(i)+".wav"
  chunk.export(archivoChunk,format = "wav")
  audioChunks.append(archivoChunk)



In [0]:
audioChunks

Out[71]: ['/tmp/chunk_0.wav', '/tmp/chunk_5000.wav', '/tmp/chunk_10000.wav']

In [0]:
idioma = "es-Es"

In [0]:
configuracion = speechsdk.SpeechConfig(
  subscription = clave,
  region = region,
  speech_recognition_language = idioma
)

In [0]:
audioChunks[0]

Out[74]: '/tmp/chunk_0.wav'

In [0]:
archivo = speechsdk.audio.AudioConfig(filename=audioChunks[0])

In [0]:
servicio = speechsdk.SpeechRecognizer(
  speech_config=configuracion,
  audio_config=archivo
)

In [0]:
resultado = servicio.recognize_once()

In [0]:
resultado.text

Out[78]: '¿Hola Santiago, cómo estás? Te habla Andrés, asesor comercial y encargado de tu servicio telefónico.'

In [0]:
textoDelAudio = ""

In [0]:
for chunk in audioChunks:
  print("Obteniendo el audio del chunk:"+chunk)
  archivo = speechsdk.audio.AudioConfig(filename = chunk)
  servicio = speechsdk.SpeechRecognizer(
    speech_config = configuracion,
    audio_config= archivo
  )
  resultado = servicio.recognize_once()
  textoDelAudio = textoDelAudio + " " + resultado.text

Obteniendo el audio del chunk:/tmp/chunk_0.wav
Obteniendo el audio del chunk:/tmp/chunk_5000.wav
Obteniendo el audio del chunk:/tmp/chunk_10000.wav


In [0]:
textoDelAudio

Out[82]: ' ¿Hola Santiago, cómo estás? Te habla Andrés, asesor comercial y encargado de tu servicio telefónico. ¿Cómo estás? Ah, bien, gracias, no, no estoy interesado. Muchísimas gracias. Mira Santiago. ¿El motivo? Mi llamado es breve y es para darte un beneficio hasta del 7% en tu factura del móvil.'

In [0]:
#Función de traducción de voz a texto
def traduccionDeVozHaciaTexto(archivoDeAudio, idioma):
  tiempoDeChunkMilisegundos = 5000
  audio = AudioSegment.from_file("/tmp/AUDIO_CALL_CENTER.wav")
  audioChunks = []
  for i in range(0,len(audio),tiempoDeChunkMilisegundos):
    chunk = audio[i:(i+tiempoDeChunkMilisegundos)]
    archivoChunk = "/tmp/chunk_"+str(i)+".wav"
    chunk.export(archivoChunk,format = "wav")
    audioChunks.append(archivoChunk)

  configuracion = speechsdk.SpeechConfig(
  subscription = clave,
  region = region,
  speech_recognition_language = idioma
  )

  textoDelAudio = ""

  for chunk in audioChunks:
    print("Obteniendo el audio del chunk:"+chunk)
    archivo = speechsdk.audio.AudioConfig(filename = chunk)
    servicio = speechsdk.SpeechRecognizer(
      speech_config = configuracion,
      audio_config= archivo
    )
    resultado = servicio.recognize_once()
    textoDelAudio = textoDelAudio + " " + resultado.text
  
  return textoDelAudio



In [0]:
#Ejecutamos la función
texto = traduccionDeVozHaciaTexto("/tmp/AUDIO_CALL_CENTER.wav","es-Es")

Obteniendo el audio del chunk:/tmp/chunk_0.wav
Obteniendo el audio del chunk:/tmp/chunk_5000.wav
Obteniendo el audio del chunk:/tmp/chunk_10000.wav


In [0]:
texto

Out[86]: ' ¿Hola Santiago, cómo estás? Te habla Andrés, asesor comercial y encargado de tu servicio telefónico. ¿Cómo estás? Ah, bien, gracias, no, no estoy interesado. Muchísimas gracias. Mira Santiago. ¿El motivo? Mi llamado es breve y es para darte un beneficio hasta del 7% en tu factura del móvil.'